## 🔧 Google Drive マウント（推奨）

モデルファイルをGoogle Driveに保存することで、次回から再ダウンロード不要になります。

**メリット:**
- ✅ モデルの永続化（5GB以上のモデルを毎回ダウンロードする必要なし）
- ✅ セットアップ時間の大幅短縮（数分→数秒）
- ✅ 複数ノートブックでモデルを共有可能

**初回のみ**: Google Driveの認証が必要です（数秒で完了）

In [ ]:
# Google Drive をマウント
from google.colab import drive
import os
from pathlib import Path

print("📂 Google Drive をマウント中...\n")

# Google Driveをマウント
drive.mount('/content/drive')

# モデル保存用ディレクトリを作成
gdrive_models = Path('/content/drive/MyDrive/ComfyUI_Models')
gdrive_models.mkdir(parents=True, exist_ok=True)

print("✅ Google Drive マウント完了！")
print(f"📁 モデル保存先: {gdrive_models}\n")

# サブディレクトリを作成（ComfyUI標準構造）
subdirs = {
    'diffusion_models': gdrive_models / 'diffusion_models',
    'text_encoders': gdrive_models / 'text_encoders', 
    'vae': gdrive_models / 'vae'
}

for name, path in subdirs.items():
    path.mkdir(parents=True, exist_ok=True)
    print(f"  📁 {name}: {path}")

print("\n💡 ヒント:")
print("  • 初回実行時のみ、モデルをダウンロードしてGoogle Driveに保存します")
print("  • 2回目以降は、Google Driveから直接読み込むため高速です")
print("  • Google Driveの容量: 15GB（無料）/ モデル合計: 約6GB")

# 🎬 ComfyUI 動画生成環境 for LyricToPrompt AI Studio

このノートブックは、Reactアプリで生成した絵コンテプロンプトから動画を生成するための環境です。

## 📋 機能
- ComfyUI の自動セットアップ
- AnimateDiff / Stable Video Diffusion による動画生成
- プロンプトのバッチ処理
- 生成動画のダウンロード

## ⚠️ 注意
- GPU ランタイムを使用してください（T4以上推奨）
- 初回セットアップに10-15分かかります

## 1️⃣ 環境セットアップ

In [ ]:
# GPU確認
!nvidia-smi

import os
import json
import subprocess
from pathlib import Path

print("✅ GPU環境を確認しました")

In [ ]:
# ComfyUI のクローンとセットアップ（依存関係の完全修正版）
print("📦 ComfyUI をインストール中...")

if not os.path.exists('/content/ComfyUI'):
    # ComfyUIをクローン
    !git clone https://github.com/comfyanonymous/ComfyUI.git
    %cd /content/ComfyUI
    
    print("\n🔧 依存関係をインストール中...")
    
    # 環境を確認してCUDAバージョンに合わせる
    import torch
    cuda_version = torch.version.cuda
    print(f"🔍 検出されたCUDAバージョン: {cuda_version}")
    
    if cuda_version and cuda_version.startswith('12.8'):
        print("📦 CUDA 12.8環境用にインストール...")
        # CUDA 12.8の場合はPyTorch 2.9.1を使用
        !pip install -q torch==2.9.1 torchvision==0.24.1 torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu128
        !pip install -q xformers==0.0.33.post2
    else:
        print("📦 CUDA 12.6環境用にインストール...")
        # CUDA 12.6の場合はPyTorch 2.9.1を使用
        # CUDA 12.6の場合もPyTorch 2.9.1に統一（xformers 0.0.33.post2との互換性のため）
        !pip install -q torch==2.9.1 torchvision==0.24.1 torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu126
        !pip install -q xformers==0.0.33.post2
    
    # その他の依存関係をインストール
    print("📦 残りの依存関係をインストール中...")
    !pip install -q pillow numpy psutil pyyaml safetensors scipy tqdm
    
    print("\n✅ 依存関係のインストール完了")
else:
    %cd /content/ComfyUI
    print("✅ ComfyUI は既にインストール済み")
    
    # 既存環境でバージョンを確認・修正
    print("\n🔍 PyTorchバージョンを確認中...")
    
    import torch
    current_version = torch.__version__
    cuda_version = torch.version.cuda
    print(f"   現在のバージョン: {current_version}")
    print(f"   CUDAバージョン: {cuda_version}")
    
    # xformersの要求バージョンを確認
    try:
        import xformers
        xformers_version = xformers.__version__
        print(f"   xformersバージョン: {xformers_version}")
        
        # xformers 0.0.33.post2はPyTorch 2.9.1を要求
        if xformers_version == "0.0.33.post2" and not current_version.startswith('2.9.1'):
            print("\n⚠️ バージョン不一致を検出、修正中...")
            print("   xformers 0.0.33.post2 → PyTorch 2.9.1 が必要")
            
            if cuda_version and cuda_version.startswith('12.8'):
                !pip install -q torch==2.9.1 torchvision==0.24.1 torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu128
            else:
                # CUDA 12.6の場合もPyTorch 2.9.1に統一（xformersとの互換性のため）
                !pip install -q torch==2.9.1 torchvision==0.24.1 torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu126
            
            print("✅ PyTorchをxformersに合わせて統一しました")
        else:
            print("✅ バージョンは互換性があります")
    except ImportError:
        print("⚠️ xformersが見つかりません、インストール中...")
        !pip install -q xformers==0.0.33.post2
        # xformersインストール後、PyTorchを2.9.1に合わせる
        if cuda_version and cuda_version.startswith('12.8'):
            !pip install -q torch==2.9.1 torchvision==0.24.1 torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu128
        else:
            !pip install -q torch==2.9.1 torchvision==0.24.1 torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu126

print("\n" + "=" * 60)
print("✅ ComfyUI セットアップ完了")
print("=" * 60)

# バージョン情報を表示（再読み込みせずに直接取得）
import sys

# 既にインポート済みのモジュールから情報を取得
import torch

print(f"\n📊 環境情報:")
print(f"   • Python: {sys.version.split()[0]}")
print(f"   • PyTorch: {torch.__version__}")
print(f"   • CUDA: {torch.version.cuda}")
print(f"   • cuDNN: {torch.backends.cudnn.version()}")

try:
    import xformers
    print(f"   • xformers: {xformers.__version__}")
    
    # 互換性チェック
    torch_major_minor = '.'.join(torch.__version__.split('.')[:2])
    if xformers.__version__ == "0.0.33.post2":
        if torch_major_minor == "2.9":
            print(f"\n✅ 互換性チェック: PyTorch {torch.__version__} ↔ xformers {xformers.__version__} OK")
        else:
            print(f"\n⚠️ 警告: PyTorch {torch.__version__} と xformers {xformers.__version__} の互換性に問題がある可能性")
except ImportError:
    print(f"   • xformers: インストール中...")

print("\n💡 すべての依存関係が正しく構成されました")
print("💡 この環境でComfyUIを使用できます")


In [ ]:
# カスタムノードのインストール（動画生成用）
print("📦 動画生成用カスタムノードをインストール中...")

%cd /content/ComfyUI/custom_nodes

# AnimateDiff-Evolved
if not os.path.exists('ComfyUI-AnimateDiff-Evolved'):
    !git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved.git

# VideoHelperSuite
if not os.path.exists('ComfyUI-VideoHelperSuite'):
    !git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
    %cd ComfyUI-VideoHelperSuite
    !pip install -q -r requirements.txt
    %cd ..

# Advanced ControlNet
if not os.path.exists('ComfyUI-Advanced-ControlNet'):
    !git clone https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet.git

%cd /content/ComfyUI
print("✅ カスタムノードのインストール完了")

In [ ]:
# モデルのダウンロードとGoogle Driveへの保存
print("📥 WAN Text-to-Video モデルのセットアップ中...\n")

import urllib.request
import shutil

# Google Driveのモデルパス
gdrive_models = Path('/content/drive/MyDrive/ComfyUI_Models')
use_gdrive = gdrive_models.exists()

if use_gdrive:
    print("✅ Google Drive マウント検出")
    print(f"📁 モデル保存先: {gdrive_models}\n")
else:
    print("⚠️ Google Drive未マウント - ローカルに保存します")
    print("💡 次回からはセル2でGoogle Driveをマウントすることを推奨\n")

# ===== 1. Diffusion Model (WAN 2.1 T2V) =====
print("1️⃣ Diffusion Model (WAN 2.1 T2V 1.3B)")

if use_gdrive:
    # Google Driveのディレクトリを作成
    gdrive_diffusion_dir = gdrive_models / 'diffusion_models'
    gdrive_diffusion_dir.mkdir(parents=True, exist_ok=True)
    
    gdrive_diffusion = gdrive_diffusion_dir / 'wan2.1_t2v_1.3B_fp16.safetensors'
    local_diffusion_dir = Path('/content/ComfyUI/models/diffusion_models')
    local_diffusion_dir.mkdir(parents=True, exist_ok=True)
    local_diffusion = local_diffusion_dir / 'wan2.1_t2v_1.3B_fp16.safetensors'
    
    if gdrive_diffusion.exists():
        print(f"   ✅ Google Drive から検出: {gdrive_diffusion.name}")
        if not local_diffusion.exists():
            print("   🔗 シンボリックリンクを作成中...")
            os.symlink(gdrive_diffusion, local_diffusion)
            print("   ✅ リンク完了（高速アクセス）")
        else:
            print("   ✅ 既にリンク済み")
    else:
        print("   📥 ダウンロード中... (約3GB)")
        !wget -q --show-progress -O {gdrive_diffusion} https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/wan2.1_t2v_1.3B_fp16.safetensors
        if gdrive_diffusion.exists():
            if not local_diffusion.exists():
                os.symlink(gdrive_diffusion, local_diffusion)
            print(f"   ✅ ダウンロード完了: {gdrive_diffusion.stat().st_size / 1e9:.2f} GB")
        else:
            print("   ❌ ダウンロードに失敗しました")
else:
    local_diffusion_dir = Path('/content/ComfyUI/models/diffusion_models')
    local_diffusion_dir.mkdir(parents=True, exist_ok=True)
    local_diffusion = local_diffusion_dir / 'wan2.1_t2v_1.3B_fp16.safetensors'
    
    if not local_diffusion.exists():
        print("   📥 ダウンロード中... (約3GB)")
        !wget -q --show-progress -O {local_diffusion} https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/wan2.1_t2v_1.3B_fp16.safetensors
        print(f"   ✅ ダウンロード完了")
    else:
        print("   ✅ 既に存在します")

# ===== 2. Text Encoder (UMT5-XXL) =====
print("\n2️⃣ Text Encoder (UMT5-XXL)")

if use_gdrive:
    # Google Driveのディレクトリを作成
    gdrive_text_encoder_dir = gdrive_models / 'text_encoders'
    gdrive_text_encoder_dir.mkdir(parents=True, exist_ok=True)
    
    gdrive_text_encoder = gdrive_text_encoder_dir / 'umt5_xxl_fp8_e4m3fn_scaled.safetensors'
    local_text_encoder_dir = Path('/content/ComfyUI/models/text_encoders')
    local_text_encoder_dir.mkdir(parents=True, exist_ok=True)
    local_text_encoder = local_text_encoder_dir / 'umt5_xxl_fp8_e4m3fn_scaled.safetensors'
    
    if gdrive_text_encoder.exists():
        print(f"   ✅ Google Drive から検出: {gdrive_text_encoder.name}")
        if not local_text_encoder.exists():
            print("   🔗 シンボリックリンクを作成中...")
            os.symlink(gdrive_text_encoder, local_text_encoder)
            print("   ✅ リンク完了")
        else:
            print("   ✅ 既にリンク済み")
    else:
        print("   📥 ダウンロード中... (約2GB)")
        !wget -q --show-progress -O {gdrive_text_encoder} https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/umt5_xxl_fp8_e4m3fn_scaled.safetensors
        if gdrive_text_encoder.exists():
            if not local_text_encoder.exists():
                os.symlink(gdrive_text_encoder, local_text_encoder)
            print(f"   ✅ ダウンロード完了: {gdrive_text_encoder.stat().st_size / 1e9:.2f} GB")
        else:
            print("   ❌ ダウンロードに失敗しました")
else:
    local_text_encoder_dir = Path('/content/ComfyUI/models/text_encoders')
    local_text_encoder_dir.mkdir(parents=True, exist_ok=True)
    local_text_encoder = local_text_encoder_dir / 'umt5_xxl_fp8_e4m3fn_scaled.safetensors'
    
    if not local_text_encoder.exists():
        print("   📥 ダウンロード中... (約2GB)")
        !wget -q --show-progress -O {local_text_encoder} https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/umt5_xxl_fp8_e4m3fn_scaled.safetensors
        print("   ✅ ダウンロード完了")
    else:
        print("   ✅ 既に存在します")

# ===== 3. VAE モデル (WAN 2.1) =====
print("\n3️⃣ VAE モデル (WAN 2.1)")

if use_gdrive:
    # Google Driveのディレクトリを作成
    gdrive_vae_dir = gdrive_models / 'vae'
    gdrive_vae_dir.mkdir(parents=True, exist_ok=True)
    
    gdrive_vae = gdrive_vae_dir / 'wan_2.1_vae.safetensors'
    local_vae_dir = Path('/content/ComfyUI/models/vae')
    local_vae_dir.mkdir(parents=True, exist_ok=True)
    local_vae = local_vae_dir / 'wan_2.1_vae.safetensors'
    
    if gdrive_vae.exists():
        print(f"   ✅ Google Drive から検出: {gdrive_vae.name}")
        if not local_vae.exists():
            print("   🔗 シンボリックリンクを作成中...")
            os.symlink(gdrive_vae, local_vae)
            print("   ✅ リンク完了")
        else:
            print("   ✅ 既にリンク済み")
    else:
        print("   📥 ダウンロード中... (約1GB)")
        !wget -q --show-progress -O {gdrive_vae} https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/wan_2.1_vae.safetensors
        if gdrive_vae.exists():
            if not local_vae.exists():
                os.symlink(gdrive_vae, local_vae)
            print(f"   ✅ ダウンロード完了: {gdrive_vae.stat().st_size / 1e9:.2f} GB")
        else:
            print("   ❌ ダウンロードに失敗しました")
else:
    local_vae_dir = Path('/content/ComfyUI/models/vae')
    local_vae_dir.mkdir(parents=True, exist_ok=True)
    local_vae = local_vae_dir / 'wan_2.1_vae.safetensors'
    
    if not local_vae.exists():
        print("   📥 ダウンロード中... (約1GB)")
        !wget -q --show-progress -O {local_vae} https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/wan_2.1_vae.safetensors
        print("   ✅ ダウンロード完了")
    else:
        print("   ✅ 既に存在します")

# ===== 4. AnimateDiff モーションモデル =====
print("\n4️⃣ AnimateDiff モーションモデル")

if use_gdrive:
    # Google Driveのディレクトリを作成
    gdrive_animatediff_dir = gdrive_models / 'animatediff_models'
    gdrive_animatediff_dir.mkdir(parents=True, exist_ok=True)
    
    gdrive_animatediff = gdrive_animatediff_dir / 'mm_sd15_v3.safetensors'
    local_animatediff_dir = Path('/content/ComfyUI/models/animatediff_models')
    local_animatediff_dir.mkdir(parents=True, exist_ok=True)
    local_animatediff = local_animatediff_dir / 'mm_sd15_v3.safetensors'
    
    if gdrive_animatediff.exists():
        print(f"   ✅ Google Drive から検出: {gdrive_animatediff.name}")
        if not local_animatediff.exists():
            print("   🔗 シンボリックリンクを作成中...")
            os.symlink(gdrive_animatediff, local_animatediff)
            print("   ✅ リンク完了")
        else:
            print("   ✅ 既にリンク済み")
    else:
        print("   📥 ダウンロード中... (約1.7GB)")
        !wget -q --show-progress -O {gdrive_animatediff} https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd15_v3.safetensors
        if gdrive_animatediff.exists():
            if not local_animatediff.exists():
                os.symlink(gdrive_animatediff, local_animatediff)
            print(f"   ✅ ダウンロード完了: {gdrive_animatediff.stat().st_size / 1e9:.2f} GB")
        else:
            print("   ❌ ダウンロードに失敗しました")
else:
    local_animatediff_dir = Path('/content/ComfyUI/models/animatediff_models')
    local_animatediff_dir.mkdir(parents=True, exist_ok=True)
    local_animatediff = local_animatediff_dir / 'mm_sd15_v3.safetensors'
    
    if not local_animatediff.exists():
        print("   📥 ダウンロード中... (約1.7GB)")
        !wget -q --show-progress -O {local_animatediff} https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd15_v3.safetensors
        print("   ✅ ダウンロード完了")
    else:
        print("   ✅ 既に存在します")

# ===== 完了サマリー =====
print("\n" + "=" * 60)
print("✅ モデルセットアップ完了！")
print("=" * 60)

if use_gdrive:
    total_size = sum([
        gdrive_diffusion.stat().st_size if gdrive_diffusion.exists() else 0,
        gdrive_text_encoder.stat().st_size if gdrive_text_encoder.exists() else 0,
        gdrive_vae.stat().st_size if gdrive_vae.exists() else 0,
        gdrive_animatediff.stat().st_size if gdrive_animatediff.exists() else 0
    ]) / 1e9
    print(f"📦 Google Drive 使用量: {total_size:.2f} GB")
    print("💡 次回以降は再ダウンロード不要です！")
else:
    print("💡 次回はセル3でGoogle Driveをマウントすることを推奨します")

## 2️⃣ ComfyUI サーバー起動

In [ ]:
# ComfyUI サーバーをバックグラウンドで起動
import subprocess
import time
import sys
from threading import Thread

print("🚀 ComfyUI サーバーを起動しています...\n")

def start_comfyui():
    """ComfyUIサーバーをバックグラウンドで起動"""
    try:
        process = subprocess.Popen(
            ['python', 'main.py', '--listen', '0.0.0.0', '--port', '8188'],
            cwd='/content/ComfyUI',
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,  # stderrもstdoutにマージ
            text=True,
            bufsize=1
        )
        
        print("📋 ComfyUIサーバーのログ:")
        print("-" * 50)
        
        # ログをリアルタイムで出力
        for line in process.stdout:
            print(line.rstrip())
            sys.stdout.flush()
            
    except Exception as e:
        print(f"❌ サーバー起動エラー: {e}")

# バックグラウンドでComfyUIを起動
print("⏳ サーバー起動スレッドを開始...")
thread = Thread(target=start_comfyui)
thread.daemon = True
thread.start()

print("⏳ サーバーの起動を待機中（30秒）...")
time.sleep(30)

print("\n" + "=" * 50)
print("🌐 Cloudflare トンネルを作成中...")
print("=" * 50 + "\n")

# Cloudflared のダウンロード
import os
if not os.path.exists('cloudflared-linux-amd64'):
    print("📥 cloudflared をダウンロード中...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64
    print("✅ ダウンロード完了")
else:
    print("✅ cloudflared は既に存在します")

# Cloudflare トンネルを起動
import re

print("\n🔗 トンネルを確立中...\n")

tunnel_process = subprocess.Popen(
    ['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8188'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,  # すべての出力を取得
    text=True,
    bufsize=1
)

# トンネルURLを取得（最大30秒待機）
comfyui_url = None
start_time = time.time()
max_wait = 30

print("📡 トンネルURL取得中（最大30秒待機）...\n")

try:
    for line in tunnel_process.stdout:
        print(line.rstrip())
        
        # trycloudflare.com を含む行を検索
        if 'trycloudflare.com' in line:
            url_match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
            if url_match:
                comfyui_url = url_match.group(0)
                print("\n" + "=" * 50)
                print("✅ ComfyUI が起動しました！")
                print("=" * 50)
                print(f"\n🌐 アクセスURL:")
                print(f"   {comfyui_url}")
                print(f"\n💡 このURLをクリックしてComfyUI WebUIを開いてください")
                print("=" * 50 + "\n")
                break
        
        # タイムアウトチェック
        if time.time() - start_time > max_wait:
            print(f"\n⚠️ {max_wait}秒経過しましたが、URLが見つかりませんでした")
            break
            
except KeyboardInterrupt:
    print("\n⚠️ ユーザーによって中断されました")

# URLが取得できなかった場合の診断
if comfyui_url is None:
    print("\n" + "=" * 50)
    print("⚠️ Cloudflare URLの自動取得に失敗しました")
    print("=" * 50)
    print("\n🔍 診断コマンドを実行中...\n")
    
    # ComfyUIプロセスの確認
    print("1️⃣ ComfyUIプロセスの確認:")
    !ps aux | grep -E 'main.py|python' | grep -v grep
    
    print("\n2️⃣ ポート8188の確認:")
    !netstat -tuln 2>/dev/null | grep 8188 || ss -tuln 2>/dev/null | grep 8188 || echo "netstat/ss コマンドが利用できません"
    
    print("\n3️⃣ ComfyUIログの最後の10行:")
    !tail -10 /content/ComfyUI/*.log 2>/dev/null || echo "ログファイルが見つかりません"
    
    print("\n💡 手動でトンネルを作成するには、新しいセルで以下を実行:")
    print("   !./cloudflared-linux-amd64 tunnel --url http://localhost:8188")
    print("\n   または、ポート8188が使用中か確認してください")
    print("=" * 50 + "\n")
else:
    print("✅ セットアップ完了！上記のURLでComfyUIにアクセスできます")
    print("\n📌 このセルは実行中のまま維持してください")
    print("   （サーバーとトンネルが動作しています）")

## 3️⃣ プロンプトから動画生成

### 🎯 ワークフロー

**セル10.5**: JSONファイルのアップロードと最適化
- 📁 Reactアプリからエクスポートした `video_prompts_for_comfyui.json` をアップロード
- 🔄 データを自動的にComfyUI用に最適化
- 📊 データサイズの比較と統計情報を表示
- 📋 シーン一覧を見やすいテーブルで表示

**セル11**: 動画生成関数の準備
- 🎬 WAN Text-to-Video ワークフローを定義

**セル12**: バッチ動画生成（進捗可視化付き）
- 🎨 リアルタイムで進捗バーを表示
- ⏱️ 経過時間と残り時間を推定
- 📈 成功/失敗の統計を表示

**セル13**: 進捗確認ダッシュボード
- 🔄 円形プログレスバーで進捗を可視化
- 📊 完成したシーン数、残りシーン数、推定時間を表示
- 📹 生成済みファイル一覧と詳細情報

### 💡 使い方のヒント
- セル10.5を実行すると、ファイル選択ダイアログが表示されます
- JSONファイルをアップロードすると、自動的に解析・最適化されます
- セル12実行中は、リアルタイムで進捗が更新されます
- セル13は何度でも実行して進捗を確認できます

In [ ]:
# Reactアプリからエクスポートしたプロンプトをここにペースト
# または、JSONファイルをアップロードして読み込み

prompts = [
    {
        "scene": 1,
        "timestamp": "0:00-0:05",
        "prompt": "cinematic shot of a young woman walking through a neon-lit city street at night, cyberpunk style, bokeh lights, 8k, high quality",
        "negative_prompt": "blurry, low quality, distorted"
    },
    {
        "scene": 2,
        "timestamp": "0:05-0:10",
        "prompt": "close-up shot of her face with reflections of neon signs, emotional expression, dramatic lighting, cinematic",
        "negative_prompt": "blurry, low quality, distorted"
    },
    # ここに追加のプロンプトを追加...
]

print(f"✅ {len(prompts)} シーンのプロンプトを読み込みました")

In [ ]:
# 📋 Reactアプリからのプロンプトを読み込み（完全対応版）

import json
from google.colab import files
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

print("=" * 70)
print("🎬 動画生成用プロンプト読み込み")
print("=" * 70)

# ==========================================
# 📁 JSONファイルをアップロード
# ==========================================
print("\n📁 JSONファイルをアップロードしてください")
print("   • Reactアプリでエクスポートした video_prompts_for_comfyui.json")
print("   • または storyboard_prompts_for_comfyui.json\n")

uploaded = files.upload()

if not uploaded:
    print("⚠️ ファイルがアップロードされませんでした")
    print("💡 下記のサンプルデータで続行します\n")
    
    # サンプルデータ
    raw_prompts = [
        {
            "scene": 1,
            "timestamp": "0:00-0:05",
            "duration_seconds": 5,
            "prompt": "A cinematic long shot captured at eye level with a handheld camera...",
            "negative_prompt": "blurry, low quality, distorted, bad anatomy, deformed, ugly, malformed, static, messy background, amateur"
        }
    ]
else:
    # アップロードされたファイルを読み込み
    filename = list(uploaded.keys())[0]
    print(f"✅ ファイルを受信: {filename}")
    
    try:
        # JSONを解析
        file_content = uploaded[filename].decode('utf-8')
        raw_prompts = json.loads(file_content)
        
        # バリデーション
        if not isinstance(raw_prompts, list):
            raise ValueError("JSONはシーンの配列である必要があります")
        
        if len(raw_prompts) == 0:
            raise ValueError("シーンが見つかりません")
        
        print(f"✅ {len(raw_prompts)} シーンのデータを読み込みました")
        
        # ファイル情報を表示
        file_size_kb = len(file_content) / 1024
        print(f"📊 ファイルサイズ: {file_size_kb:.2f} KB")
        
        # データ構造を分析
        first_scene = raw_prompts[0]
        field_count = len(first_scene.keys())
        has_japanese = 'cut_description' in first_scene or 'direction_memo' in first_scene
        
        print(f"📋 データ構造:")
        print(f"   • フィールド数: {field_count}")
        print(f"   • 形式: {'絵コンテ（日本語混在）' if has_japanese else '動画プロンプト（英語のみ）'}")
        
    except json.JSONDecodeError as e:
        print(f"❌ JSONの解析に失敗しました: {e}")
        print("💡 ファイルが正しいJSON形式か確認してください")
        raw_prompts = []
    except Exception as e:
        print(f"❌ エラー: {e}")
        raw_prompts = []

# ==========================================
# 🔄 ComfyUI用に最適化
# ==========================================
if raw_prompts:
    print("\n" + "=" * 70)
    print("🔄 ComfyUI用にデータを最適化中...")
    print("=" * 70 + "\n")
    
    prompts = []
    skipped_fields = set()
    
    for idx, raw_prompt in enumerate(raw_prompts, 1):
        # 必須フィールドをチェック
        if 'scene' not in raw_prompt or 'prompt' not in raw_prompt:
            print(f"⚠️ シーン {idx}: 必須フィールドが不足しています")
            continue
        
        # 不要なフィールドを記録
        for key in raw_prompt.keys():
            if key not in ['scene', 'timestamp', 'prompt', 'negative_prompt', 'duration_seconds']:
                skipped_fields.add(key)
        
        # 最適化されたデータを作成
        optimized = {
            'scene': raw_prompt['scene'],
            'timestamp': raw_prompt.get('timestamp', f"0:00-0:05"),
            'prompt': raw_prompt['prompt'],
            'negative_prompt': raw_prompt.get('negative_prompt', 
                'blurry, low quality, distorted, bad anatomy, deformed, ugly, malformed, static, messy background, amateur')
        }
        prompts.append(optimized)
    
    # ==========================================
    # ✅ 結果表示（詳細な可視化）
    # ==========================================
    print(f"✅ {len(prompts)} シーンのプロンプトを最適化しました\n")
    
    # データサイズの比較
    original_size = len(json.dumps(raw_prompts, ensure_ascii=False))
    optimized_size = len(json.dumps(prompts, ensure_ascii=False))
    reduction_percent = (1 - optimized_size / original_size) * 100
    
    print(f"📊 データサイズの比較:")
    print(f"   元のサイズ:     {original_size / 1024:.2f} KB")
    print(f"   最適化後:       {optimized_size / 1024:.2f} KB")
    print(f"   削減率:         {reduction_percent:.1f}%")
    
    if skipped_fields:
        print(f"\n🗑️  除外されたフィールド:")
        for field in sorted(skipped_fields):
            print(f"   • {field}")
    
    # シーン一覧を表示（HTMLテーブル）
    print("\n" + "=" * 70)
    print("📋 読み込まれたシーン一覧")
    print("=" * 70 + "\n")
    
    # HTMLテーブルを作成
    html_table = """
    <style>
        .scene-table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        .scene-table thead {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
        }
        .scene-table th {
            padding: 12px;
            text-align: left;
            font-weight: 600;
        }
        .scene-table td {
            padding: 10px 12px;
            border-bottom: 1px solid #e0e0e0;
        }
        .scene-table tr:hover {
            background-color: #f5f5f5;
        }
        .scene-number {
            font-weight: 700;
            color: #667eea;
            font-size: 16px;
        }
        .timestamp {
            color: #7f8c8d;
            font-family: 'Courier New', monospace;
        }
        .prompt-preview {
            max-width: 400px;
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;
            color: #2c3e50;
        }
        .status-ready {
            color: #27ae60;
            font-weight: 600;
        }
    </style>
    <table class="scene-table">
        <thead>
            <tr>
                <th>シーン</th>
                <th>タイムスタンプ</th>
                <th>プロンプト（プレビュー）</th>
                <th>ステータス</th>
            </tr>
        </thead>
        <tbody>
    """
    
    for prompt in prompts:
        prompt_preview = prompt['prompt'][:80] + "..." if len(prompt['prompt']) > 80 else prompt['prompt']
        html_table += f"""
        <tr>
            <td class="scene-number">#{prompt['scene']}</td>
            <td class="timestamp">{prompt['timestamp']}</td>
            <td class="prompt-preview">{prompt_preview}</td>
            <td class="status-ready">✓ 準備完了</td>
        </tr>
        """
    
    html_table += """
        </tbody>
    </table>
    """
    
    display(HTML(html_table))
    
    # 統計情報を表示
    total_prompts = len(prompts)
    avg_prompt_length = sum(len(p['prompt']) for p in prompts) / total_prompts
    
    # タイムスタンプから総時間を計算
    if prompts:
        last_timestamp = prompts[-1]['timestamp']
        if '-' in last_timestamp:
            end_time = last_timestamp.split('-')[1]
            if ':' in end_time:
                parts = end_time.split(':')
                total_seconds = int(parts[0]) * 60 + int(parts[1])
                total_minutes = total_seconds // 60
                remaining_seconds = total_seconds % 60
                
                print(f"\n📈 統計情報:")
                print(f"   • 総シーン数:        {total_prompts}")
                print(f"   • 推定動画長:        {total_minutes}分{remaining_seconds}秒")
                print(f"   • 平均プロンプト長:  {avg_prompt_length:.0f} 文字")
                print(f"   • 推定生成時間:      {total_prompts * 2}〜{total_prompts * 3} 分")
    
    print("\n" + "=" * 70)
    print("💡 次のステップ:")
    print("=" * 70)
    print("   1️⃣  セル11を実行 → 動画生成関数を準備")
    print("   2️⃣  セル12を実行 → 全シーンの動画を生成")
    print("   3️⃣  セル13を実行 → 進捗確認（何度でも実行可能）")
    print("=" * 70 + "\n")
else:
    print("\n❌ プロンプトの読み込みに失敗しました")
    print("💡 ファイルを確認して再度実行してください")
    prompts = []


In [ ]:
# ComfyUI API を使った動画生成関数（WAN Text-to-Video対応）
import requests
import uuid
import time
from IPython.display import Video, display

def generate_video_from_prompt(prompt_data, output_dir='/content/outputs'):
    """
    WAN Text-to-Videoモデルを使用してプロンプトから動画を生成
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # WAN Text-to-Video ワークフローの定義
    workflow = {
        "37": {  # UNET Loader (WAN 2.1 T2V)
            "inputs": {
                "unet_name": "wan2.1_t2v_1.3B_fp16.safetensors",
                "weight_dtype": "default"
            },
            "class_type": "UNETLoader"
        },
        "38": {  # CLIP Loader (UMT5-XXL)
            "inputs": {
                "clip_name": "umt5_xxl_fp8_e4m3fn_scaled.safetensors",
                "type": "wan",
                "decoder_dtype": "default"
            },
            "class_type": "CLIPLoader"
        },
        "39": {  # VAE Loader
            "inputs": {
                "vae_name": "wan_2.1_vae.safetensors"
            },
            "class_type": "VAELoader"
        },
        "6": {  # CLIP Text Encode (Positive)
            "inputs": {
                "text": prompt_data['prompt'],
                "clip": ["38", 0]
            },
            "class_type": "CLIPTextEncode"
        },
        "7": {  # CLIP Text Encode (Negative)
            "inputs": {
                "text": prompt_data.get('negative_prompt', 'blurry, low quality, distorted, static, messy background'),
                "clip": ["38", 0]
            },
            "class_type": "CLIPTextEncode"
        },
        "40": {  # Empty Hunyuan Latent Video
            "inputs": {
                "width": 832,
                "height": 480,
                "length": 33,  # 33フレーム
                "batch_size": 1
            },
            "class_type": "EmptyHunyuanLatentVideo"
        },
        "48": {  # Model Sampling SD3
            "inputs": {
                "shift": 8.0,
                "model": ["37", 0]
            },
            "class_type": "ModelSamplingSD3"
        },
        "3": {  # KSampler
            "inputs": {
                "seed": int(time.time() * 1000) % (2**32),
                "steps": 30,
                "cfg": 6.0,
                "sampler_name": "uni_pc",
                "scheduler": "simple",
                "denoise": 1.0,
                "model": ["48", 0],
                "positive": ["6", 0],
                "negative": ["7", 0],
                "latent_image": ["40", 0]
            },
            "class_type": "KSampler"
        },
        "8": {  # VAE Decode
            "inputs": {
                "samples": ["3", 0],
                "vae": ["39", 0]
            },
            "class_type": "VAEDecode"
        },
        "28": {  # Save Animated WEBP
            "inputs": {
                "filename_prefix": f"scene_{prompt_data['scene']:02d}",
                "fps": 16,
                "lossless": False,
                "quality": 90,
                "method": "default",
                "images": ["8", 0]
            },
            "class_type": "SaveAnimatedWEBP"
        }
    }
    
    # ワークフローを送信
    client_id = str(uuid.uuid4())
    payload = {
        "prompt": workflow,
        "client_id": client_id
    }
    
    try:
        response = requests.post(
            'http://localhost:8188/prompt',
            json=payload,
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            prompt_id = result.get('prompt_id')
            print(f"✅ シーン {prompt_data['scene']} の生成を開始しました")
            print(f"   Prompt ID: {prompt_id}")
            print(f"   解像度: 832x480, フレーム数: 33, FPS: 16")
            return prompt_id
        else:
            print(f"❌ エラー: {response.status_code}")
            print(f"   レスポンス: {response.text}")
            return None
            
    except requests.exceptions.Timeout:
        print(f"❌ タイムアウト: サーバーが応答しません")
        return None
    except Exception as e:
        print(f"❌ 生成エラー: {e}")
        return None

print("✅ WAN Text-to-Video 動画生成関数を準備しました")
print("📊 設定:")
print("   • モデル: WAN 2.1 T2V 1.3B")
print("   • CLIP: UMT5-XXL")
print("   • 解像度: 832x480")
print("   • フレーム数: 33 (約2秒@16fps)")
print("   • サンプラー: uni_pc (30 steps)")
print("   • CFG: 6.0")

In [ ]:
# 全シーンの動画を生成（詳細な進捗可視化付き）
import time
from datetime import datetime
from IPython.display import display, HTML, clear_output

print("=" * 70)
print("🎬 動画生成を開始します")
print("=" * 70)
print(f"⏰ 開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 総シーン数: {len(prompts)}")
print(f"⏱️  推定所要時間: {len(prompts) * 2}〜{len(prompts) * 3} 分")
print("=" * 70 + "\n")

generated_ids = []
failed_scenes = []
start_time = time.time()

# 進捗バーのHTMLテンプレート
def create_progress_html(current, total, scene_num, status, elapsed_time):
    progress_percent = (current / total) * 100
    
    # 残り時間を推定
    if current > 0:
        avg_time_per_scene = elapsed_time / current
        remaining_scenes = total - current
        estimated_remaining = avg_time_per_scene * remaining_scenes
        remaining_minutes = int(estimated_remaining // 60)
        remaining_seconds = int(estimated_remaining % 60)
        time_estimate = f"{remaining_minutes}分{remaining_seconds}秒"
    else:
        time_estimate = "計算中..."
    
    elapsed_minutes = int(elapsed_time // 60)
    elapsed_seconds = int(elapsed_time % 60)
    
    html = f"""
    <style>
        .progress-container {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 10px;
            color: white;
            margin: 10px 0;
        }}
        .progress-header {{
            display: flex;
            justify-content: space-between;
            margin-bottom: 10px;
            font-size: 18px;
            font-weight: 600;
        }}
        .progress-bar-bg {{
            width: 100%;
            height: 30px;
            background: rgba(255,255,255,0.3);
            border-radius: 15px;
            overflow: hidden;
            position: relative;
        }}
        .progress-bar-fill {{
            height: 100%;
            background: linear-gradient(90deg, #4CAF50, #8BC34A);
            width: {progress_percent}%;
            transition: width 0.3s ease;
            border-radius: 15px;
        }}
        .progress-text {{
            position: absolute;
            width: 100%;
            text-align: center;
            line-height: 30px;
            font-weight: 700;
            color: white;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.3);
        }}
        .status-grid {{
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            gap: 15px;
            margin-top: 15px;
        }}
        .status-item {{
            background: rgba(255,255,255,0.2);
            padding: 10px;
            border-radius: 8px;
        }}
        .status-label {{
            font-size: 12px;
            opacity: 0.8;
            margin-bottom: 5px;
        }}
        .status-value {{
            font-size: 20px;
            font-weight: 700;
        }}
        .current-scene {{
            background: rgba(255,255,255,0.3);
            padding: 15px;
            border-radius: 8px;
            margin-top: 15px;
            border-left: 4px solid #4CAF50;
        }}
    </style>
    <div class="progress-container">
        <div class="progress-header">
            <span>🎬 動画生成進捗</span>
            <span>{current} / {total} シーン</span>
        </div>
        <div class="progress-bar-bg">
            <div class="progress-bar-fill"></div>
            <div class="progress-text">{progress_percent:.1f}%</div>
        </div>
        <div class="status-grid">
            <div class="status-item">
                <div class="status-label">⏱️ 経過時間</div>
                <div class="status-value">{elapsed_minutes}分{elapsed_seconds}秒</div>
            </div>
            <div class="status-item">
                <div class="status-label">⏳ 残り時間（推定）</div>
                <div class="status-value">{time_estimate}</div>
            </div>
            <div class="status-item">
                <div class="status-label">✅ 完了</div>
                <div class="status-value">{len(generated_ids)}</div>
            </div>
            <div class="status-item">
                <div class="status-label">❌ 失敗</div>
                <div class="status-value">{len(failed_scenes)}</div>
            </div>
        </div>
        <div class="current-scene">
            <strong>📹 現在処理中:</strong> シーン #{scene_num}<br>
            <strong>🔄 ステータス:</strong> {status}
        </div>
    </div>
    """
    return html

# 各シーンを処理
for idx, prompt_data in enumerate(prompts, 1):
    current_time = time.time() - start_time
    
    # 進捗表示を更新
    clear_output(wait=True)
    display(HTML(create_progress_html(
        idx - 1, 
        len(prompts), 
        prompt_data['scene'], 
        "リクエスト送信中...",
        current_time
    )))
    
    print(f"\n{'='*70}")
    print(f"📹 シーン {prompt_data['scene']} / {len(prompts)}")
    print(f"{'='*70}")
    print(f"⏰ 時刻: {datetime.now().strftime('%H:%M:%S')}")
    print(f"🕐 タイムスタンプ: {prompt_data['timestamp']}")
    print(f"📝 プロンプト: {prompt_data['prompt'][:100]}...")
    print(f"{'='*70}")
    
    try:
        prompt_id = generate_video_from_prompt(prompt_data)
        
        if prompt_id:
            generated_ids.append({
                'scene': prompt_data['scene'],
                'prompt_id': prompt_id,
                'timestamp': prompt_data['timestamp']
            })
            print(f"✅ シーン {prompt_data['scene']} のリクエストを送信しました")
        else:
            failed_scenes.append(prompt_data['scene'])
            print(f"❌ シーン {prompt_data['scene']} のリクエストに失敗しました")
    
    except Exception as e:
        failed_scenes.append(prompt_data['scene'])
        print(f"❌ エラー (シーン {prompt_data['scene']}): {e}")
    
    # 過負荷を防ぐため待機
    if idx < len(prompts):
        print(f"⏸️  次のシーンまで2秒待機...")
        time.sleep(2)

# 最終結果を表示
total_time = time.time() - start_time
total_minutes = int(total_time // 60)
total_seconds = int(total_time % 60)

clear_output(wait=True)

# 最終進捗バー
display(HTML(create_progress_html(
    len(prompts), 
    len(prompts), 
    "完了", 
    "全シーン送信完了",
    total_time
)))

print("\n" + "=" * 70)
print("✅ 動画生成リクエストの送信が完了しました")
print("=" * 70)
print(f"📊 送信結果:")
print(f"   • 成功: {len(generated_ids)} シーン")
print(f"   • 失敗: {len(failed_scenes)} シーン")
print(f"   • 合計処理時間: {total_minutes}分{total_seconds}秒")
print(f"⏰ 完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

if failed_scenes:
    print(f"\n⚠️ 失敗したシーン: {', '.join(map(str, failed_scenes))}")
    print("💡 これらのシーンは手動で再試行できます")

# 生成中の動画の推定完了時間
if generated_ids:
    estimated_total_generation_time = len(generated_ids) * 2.5  # 平均2.5分/シーン
    estimated_completion = datetime.fromtimestamp(time.time() + estimated_total_generation_time * 60)
    
    print(f"\n📈 動画生成状況:")
    print(f"   • 生成キュー: {len(generated_ids)} シーン")
    print(f"   • 推定生成時間: {int(estimated_total_generation_time)} 分")
    print(f"   • 推定完了時刻: {estimated_completion.strftime('%H:%M:%S')}")
    print(f"\n💡 セル13を実行して進捗を確認できます")

print("\n" + "=" * 70)
print("⏳ ComfyUIサーバーで動画を生成中...")
print("💡 数分後にセル13で進捗を確認してください")
print("=" * 70 + "\n")


In [ ]:
# 🔍 生成進捗を確認（詳細な可視化付き）
import glob
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML, clear_output
import time

output_dir = '/content/ComfyUI/output'

# リアルタイム更新用の関数
def display_progress():
    video_files = sorted(glob.glob(f'{output_dir}/*.webp'))
    completed = len(video_files)
    total = len(prompts) if 'prompts' in globals() else 19
    progress_percent = (completed / total) * 100
    
    # 残り時間を推定
    if completed > 0 and completed < total:
        remaining_scenes = total - completed
        estimated_time_min = remaining_scenes * 2
        estimated_time_max = remaining_scenes * 3
        time_estimate = f"{estimated_time_min}〜{estimated_time_max}分"
    elif completed == total:
        time_estimate = "完了"
    else:
        time_estimate = "計算中..."
    
    # 進捗バーのHTML
    html = f"""
    <style>
        .progress-dashboard {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 25px;
            border-radius: 12px;
            color: white;
            margin: 20px 0;
        }}
        .dashboard-header {{
            font-size: 24px;
            font-weight: 700;
            margin-bottom: 20px;
            text-align: center;
        }}
        .progress-circle {{
            display: flex;
            justify-content: center;
            align-items: center;
            margin: 20px 0;
        }}
        .circle-progress {{
            width: 150px;
            height: 150px;
            border-radius: 50%;
            background: conic-gradient(
                #4CAF50 0% {progress_percent}%,
                rgba(255,255,255,0.3) {progress_percent}% 100%
            );
            display: flex;
            justify-content: center;
            align-items: center;
            position: relative;
        }}
        .circle-inner {{
            width: 120px;
            height: 120px;
            border-radius: 50%;
            background: white;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            color: #667eea;
        }}
        .circle-percent {{
            font-size: 32px;
            font-weight: 700;
        }}
        .circle-label {{
            font-size: 14px;
            opacity: 0.8;
        }}
        .stats-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-top: 20px;
        }}
        .stat-card {{
            background: rgba(255,255,255,0.2);
            padding: 15px;
            border-radius: 10px;
            text-align: center;
        }}
        .stat-icon {{
            font-size: 32px;
            margin-bottom: 10px;
        }}
        .stat-value {{
            font-size: 28px;
            font-weight: 700;
            margin-bottom: 5px;
        }}
        .stat-label {{
            font-size: 14px;
            opacity: 0.9;
        }}
        .file-list {{
            background: rgba(255,255,255,0.2);
            padding: 15px;
            border-radius: 10px;
            margin-top: 20px;
            max-height: 300px;
            overflow-y: auto;
        }}
        .file-item {{
            background: rgba(255,255,255,0.3);
            padding: 10px;
            margin: 5px 0;
            border-radius: 6px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }}
        .file-name {{
            font-weight: 600;
        }}
        .file-size {{
            opacity: 0.8;
            font-size: 14px;
        }}
    </style>
    <div class="progress-dashboard">
        <div class="dashboard-header">
            🎬 動画生成進捗ダッシュボード
        </div>
        <div class="progress-circle">
            <div class="circle-progress">
                <div class="circle-inner">
                    <div class="circle-percent">{progress_percent:.1f}%</div>
                    <div class="circle-label">{completed}/{total}</div>
                </div>
            </div>
        </div>
        <div class="stats-grid">
            <div class="stat-card">
                <div class="stat-icon">✅</div>
                <div class="stat-value">{completed}</div>
                <div class="stat-label">完成したシーン</div>
            </div>
            <div class="stat-card">
                <div class="stat-icon">⏳</div>
                <div class="stat-value">{total - completed}</div>
                <div class="stat-label">残りのシーン</div>
            </div>
            <div class="stat-card">
                <div class="stat-icon">⏱️</div>
                <div class="stat-value">{time_estimate}</div>
                <div class="stat-label">推定残り時間</div>
            </div>
        </div>
    """
    
    if video_files:
        total_size = sum(Path(f).stat().st_size for f in video_files) / (1024 * 1024)
        html += f"""
        <div class="file-list">
            <div style="font-size: 18px; font-weight: 600; margin-bottom: 10px;">
                📹 完成済みファイル ({len(video_files)}個, 合計 {total_size:.2f} MB)
            </div>
        """
        
        for video_path in video_files[-10:]:  # 最新10件を表示
            video_name = Path(video_path).name
            file_size = Path(video_path).stat().st_size / (1024 * 1024)
            html += f"""
            <div class="file-item">
                <span class="file-name">✓ {video_name}</span>
                <span class="file-size">{file_size:.2f} MB</span>
            </div>
            """
        
        if len(video_files) > 10:
            html += f"<div style='text-align: center; opacity: 0.8; margin-top: 10px;'>... 他 {len(video_files) - 10} ファイル</div>"
        
        html += "</div>"
    else:
        html += """
        <div style="text-align: center; padding: 20px; opacity: 0.8;">
            ⚠️ まだファイルが生成されていません<br>
            💡 数分待ってから再度このセルを実行してください
        </div>
        """
    
    html += """
        <div style="text-align: center; margin-top: 20px; opacity: 0.9;">
            ⏰ 最終更新: """ + datetime.now().strftime('%Y-%m-%d %H:%M:%S') + """
        </div>
    </div>
    """
    
    return html

# 進捗表示
print("=" * 70)
print("🔍 動画生成の進捗を確認中...")
print("=" * 70 + "\n")

display(HTML(display_progress()))

video_files = sorted(glob.glob(f'{output_dir}/*.webp'))

if video_files:
    print("\n" + "=" * 70)
    print("📊 詳細情報")
    print("=" * 70)
    
    # ファイルごとの詳細
    print("\n📹 生成済みファイル一覧:")
    for video_path in video_files:
        video_name = Path(video_path).name
        file_size = Path(video_path).stat().st_size / (1024 * 1024)
        modified_time = datetime.fromtimestamp(Path(video_path).stat().st_mtime)
        print(f"   • {video_name}")
        print(f"     サイズ: {file_size:.2f} MB | 生成時刻: {modified_time.strftime('%H:%M:%S')}")
    
    # 統計情報
    total_size = sum(Path(f).stat().st_size for f in video_files) / (1024 * 1024)
    avg_size = total_size / len(video_files)
    
    print(f"\n📈 統計:")
    print(f"   • 完成: {len(video_files)} / {len(prompts) if 'prompts' in globals() else 19} シーン")
    print(f"   • 合計サイズ: {total_size:.2f} MB")
    print(f"   • 平均サイズ: {avg_size:.2f} MB/シーン")
    
    # 残り時間の推定
    total = len(prompts) if 'prompts' in globals() else 19
    if len(video_files) < total:
        remaining = total - len(video_files)
        estimated_time_min = remaining * 2
        estimated_time_max = remaining * 3
        print(f"\n⏱️  推定残り時間: {estimated_time_min}〜{estimated_time_max}分")
        print(f"💡 このセルは何度でも実行して進捗確認できます")
    else:
        print(f"\n🎉 全シーンの生成が完了しました！")
        print(f"💡 セル14でアニメーションをプレビューできます")
        print(f"💡 セル16でZIPダウンロード、セル17でMP4変換が可能です")
else:
    print("\n⚠️ まだファイルが生成されていません")
    print("💡 数分待ってから再度このセルを実行してください")
    print("💡 セル8でComfyUIサーバーが起動しているか確認してください")

print("\n" + "=" * 70 + "\n")


## 4️⃣ 生成結果の確認

In [ ]:
# 生成された動画を確認（WEBP形式）
import glob
from IPython.display import HTML, display
import base64

output_dir = '/content/ComfyUI/output'
video_files = sorted(glob.glob(f'{output_dir}/*.webp'))

print(f"✅ {len(video_files)} 個のアニメーション画像が生成されました\n")
print("📊 WAN Text-to-Video 出力:")
print("   • 形式: Animated WEBP")
print("   • 解像度: 832x480")
print("   • フレーム数: 33")
print("   • FPS: 16\n")
print("=" * 60)

for video_path in video_files:
    video_name = Path(video_path).name
    file_size = Path(video_path).stat().st_size / (1024 * 1024)  # MB
    
    print(f"\n📹 {video_name} ({file_size:.2f} MB)")
    
    # WEBPアニメーションをBase64エンコードしてHTMLで表示
    with open(video_path, 'rb') as f:
        webp_data = f.read()
        webp_base64 = base64.b64encode(webp_data).decode('utf-8')
    
    html = f'''
    <div style="margin: 20px 0; text-align: center;">
        <img src="data:image/webp;base64,{webp_base64}" 
             style="width: 832px; height: auto; border: 2px solid #333; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    </div>
    '''
    display(HTML(html))
    
    print("\n" + "="*60)

## 5️⃣ 動画のダウンロード

In [ ]:
# すべてのアニメーション画像をZIPでダウンロード
import zipfile
from google.colab import files

output_dir = '/content/ComfyUI/output'
video_files = sorted(glob.glob(f'{output_dir}/*.webp'))

if not video_files:
    print("❌ ダウンロードする動画がありません")
    print("💡 先にセル12を実行して動画を生成してください")
else:
    zip_path = '/content/generated_videos_wan.zip'
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for video_path in video_files:
            zipf.write(video_path, Path(video_path).name)
    
    zip_size = Path(zip_path).stat().st_size / (1024 * 1024)  # MB
    
    print(f"✅ {len(video_files)} 個のアニメーション画像をZIPに圧縮しました")
    print(f"📦 ファイルサイズ: {zip_size:.2f} MB")
    print("📥 ダウンロードを開始します...")
    
    files.download(zip_path)
    
    print("\n💡 ヒント:")
    print("   • WEBP形式は多くのブラウザで再生可能です")
    print("   • MP4に変換したい場合はセル17を実行してください")

## 6️⃣ 複数動画の結合（オプション）

In [ ]:
# WEBPをMP4に変換して1つの動画に結合
!pip install -q moviepy pillow

from moviepy.editor import ImageSequenceClip, concatenate_videoclips
from PIL import Image
import numpy as np

output_dir = '/content/ComfyUI/output'
video_files = sorted(glob.glob(f'{output_dir}/*.webp'))

if not video_files:
    print("❌ 変換する動画がありません")
else:
    print("🎞️ WEBP → MP4 変換中...\n")
    
    clips = []
    
    for idx, video_path in enumerate(video_files, 1):
        print(f"[{idx}/{len(video_files)}] {Path(video_path).name} を処理中...")
        
        # WEBPアニメーションをフレームに分解
        img = Image.open(video_path)
        frames = []
        
        try:
            while True:
                # フレームをnumpy配列に変換
                frame = np.array(img.convert('RGB'))
                frames.append(frame)
                img.seek(img.tell() + 1)
        except EOFError:
            pass  # すべてのフレームを読み込み完了
        
        if frames:
            # フレームからクリップを作成（16 FPS）
            clip = ImageSequenceClip(frames, fps=16)
            clips.append(clip)
            print(f"   ✅ {len(frames)} フレームを変換しました")
    
    if clips:
        print(f"\n🔗 {len(clips)} 個のクリップを結合中...")
        
        final_clip = concatenate_videoclips(clips, method="compose")
        final_output = '/content/final_music_video_wan.mp4'
        
        final_clip.write_videofile(
            final_output,
            fps=16,
            codec='libx264',
            audio=False,
            preset='medium',
            bitrate='5000k'
        )
        
        duration = final_clip.duration
        file_size = Path(final_output).stat().st_size / (1024 * 1024)
        
        print(f"\n✅ 変換・結合完了！")
        print(f"📹 最終動画: {final_output}")
        print(f"⏱️  再生時間: {duration:.1f}秒")
        print(f"📦 ファイルサイズ: {file_size:.2f} MB")
        
        # プレビュー
        from IPython.display import Video
        display(Video(final_output, width=832, height=480))
        
        # ダウンロード
        print("\n📥 ダウンロードを開始します...")
        files.download(final_output)
    else:
        print("❌ 結合するクリップがありません")

## 🧹 ワークスペースのクリーンアップ（オプション）

次回の生成に備えて、古いファイルを削除してディスク容量を確保します。

In [ ]:
# 🧹 出力ファイルのクリーンアップ
import shutil
import glob
from pathlib import Path

print("🧹 ワークスペースをクリーンアップ中...\n")

# クリーンアップ対象
cleanup_targets = {
    'WEBP動画': '/content/ComfyUI/output/*.webp',
    '一時ファイル': '/content/ComfyUI/temp/*',
    'MP4動画': '/content/final_music_video_wan.mp4',
    'ZIPファイル': '/content/generated_videos_wan.zip'
}

total_freed_space = 0

for name, pattern in cleanup_targets.items():
    files = glob.glob(pattern)
    
    if files:
        print(f"📁 {name}:")
        space_freed = 0
        
        for file_path in files:
            try:
                file_size = Path(file_path).stat().st_size / (1024 * 1024)  # MB
                
                if Path(file_path).is_file():
                    Path(file_path).unlink()
                    print(f"   ✅ 削除: {Path(file_path).name} ({file_size:.2f} MB)")
                    space_freed += file_size
                elif Path(file_path).is_dir():
                    shutil.rmtree(file_path)
                    print(f"   ✅ 削除: {Path(file_path).name}/ (ディレクトリ)")
                    
            except Exception as e:
                print(f"   ⚠️ エラー: {Path(file_path).name} - {e}")
        
        if space_freed > 0:
            print(f"   💾 解放容量: {space_freed:.2f} MB\n")
            total_freed_space += space_freed
    else:
        print(f"📁 {name}: クリーンアップ対象なし\n")

print("=" * 60)
print(f"✅ クリーンアップ完了！")
print(f"💾 合計解放容量: {total_freed_space:.2f} MB")
print("=" * 60)

print("\n💡 ヒント:")
print("  • モデルファイルは削除されません（Google Driveに保存）")
print("  • 次回の生成前にこのセルを実行すると容量を確保できます")
print("  • 重要なファイルは事前にダウンロードしておいてください")

In [ ]:
# 📊 ストレージ使用状況の確認
import subprocess
import glob
from pathlib import Path

print("📊 ストレージ使用状況\n")
print("=" * 60)

# ディスク全体の使用状況
print("💽 ディスク全体:")
result = subprocess.run(['df', '-h', '/content'], capture_output=True, text=True)
lines = result.stdout.strip().split('\n')
if len(lines) >= 2:
    header = lines[0]
    data = lines[1]
    print(f"   {header}")
    print(f"   {data}")

print("\n" + "=" * 60)

# 各ディレクトリのサイズ
directories = {
    'ComfyUI本体': '/content/ComfyUI',
    'カスタムノード': '/content/ComfyUI/custom_nodes',
    'モデル (ローカル)': '/content/ComfyUI/models',
    '出力ファイル': '/content/ComfyUI/output',
    '一時ファイル': '/content/ComfyUI/temp'
}

print("\n📁 ディレクトリ別サイズ:\n")

for name, path in directories.items():
    if Path(path).exists():
        result = subprocess.run(['du', '-sh', path], capture_output=True, text=True)
        size = result.stdout.split('\t')[0] if result.stdout else 'N/A'
        print(f"   {name}: {size}")
    else:
        print(f"   {name}: (存在しません)")

print("\n" + "=" * 60)

# 出力ファイルの詳細
output_dir = '/content/ComfyUI/output'
if Path(output_dir).exists():
    video_files = glob.glob(f'{output_dir}/*.webp')
    
    if video_files:
        print(f"\n📹 出力ファイル ({len(video_files)}個):\n")
        total_size = 0
        
        for video_path in sorted(video_files):
            video_name = Path(video_path).name
            file_size = Path(video_path).stat().st_size / (1024 * 1024)
            total_size += file_size
            print(f"   • {video_name}: {file_size:.2f} MB")
        
        print(f"\n   合計サイズ: {total_size:.2f} MB")
    else:
        print("\n📹 出力ファイル: なし")

print("\n" + "=" * 60)

# Google Driveの使用状況
gdrive_models = Path('/content/drive/MyDrive/ComfyUI_Models')
if gdrive_models.exists():
    print("\n☁️ Google Drive (ComfyUI_Models):\n")
    
    model_files = {
        'Diffusion Model': gdrive_models / 'diffusion_models' / 'wan2.1_t2v_1.3B_fp16.safetensors',
        'Text Encoder': gdrive_models / 'text_encoders' / 'umt5_xxl_fp8_e4m3fn_scaled.safetensors',
        'VAE': gdrive_models / 'vae' / 'wan_2.1_vae.safetensors'
    }
    
    gdrive_total = 0
    
    for name, path in model_files.items():
        if path.exists():
            size = path.stat().st_size / (1024 ** 3)  # GB
            gdrive_total += size
            print(f"   • {name}: {size:.2f} GB")
        else:
            print(f"   • {name}: (未ダウンロード)")
    
    print(f"\n   合計: {gdrive_total:.2f} GB / 15 GB")
    print(f"   空き容量: {15 - gdrive_total:.2f} GB")
else:
    print("\n☁️ Google Drive: マウントされていません")

print("\n" + "=" * 60)
print("\n💡 ストレージが不足している場合:")
print("  1. 次のセル(セル19)でクリーンアップを実行")
print("  2. 不要なファイルを手動で削除")
print("  3. Google Colabセッションを再起動")

## 📝 使い方まとめ

### 🔧 0. Google Driveマウント（推奨・初回のみ）
- **セル2**: Google Driveをマウント（認証が必要）
- **メリット**: モデルを永続化して次回から高速セットアップ
- **容量**: 約6GB（Google Drive無料枠15GBで十分）

### 1️⃣ セットアップ（初回のみ）
- **セル4**: GPU環境確認
- **セル5**: ComfyUIインストール（PyTorch依存関係を自動修正）
- **セル6**: カスタムノードインストール
- **セル7**: WAN Text-to-Videoモデルのダウンロード
  - 初回: Google Driveにダウンロード（約6GB、5-10分）
  - 2回目以降: Google Driveから読み込み（数秒）

### 2️⃣ ComfyUI起動
- **セル8**: サーバーを起動してCloudflare URLを取得
- 表示されたURLでWebUIにもアクセス可能
- **重要**: このセルは実行中のまま維持してください

### 3️⃣ 動画生成（改良版）
- **セル10.5**: 📁 JSONファイルをアップロード（ドラッグ&ドロップ可能）
  - Reactアプリからエクスポートした `video_prompts_for_comfyui.json`
  - 自動的にデータを最適化
  - シーン一覧を表示
- **セル11**: WAN Text-to-Video動画生成関数を準備
- **セル12**: 全シーンの動画を生成（リアルタイム進捗表示付き）
- **セル13**: 進捗確認ダッシュボード（何度でも実行可能）

### 4️⃣ 結果の確認とダウンロード
- **セル14**: 生成されたWEBPアニメーションをプレビュー
- **セル16**: ZIPでダウンロード
- **セル17**: （オプション）WEBP→MP4変換＆結合

---

## 🔧 トラブルシューティング

### ❌ PyTorch / xformers 依存関係エラー

**エラー例1**: PyTorchバージョン不一致
```
ERROR: torchaudio 2.9.0+cu126 requires torch==2.9.0, but you have torch 2.9.1
ERROR: torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.9.1
```

**エラー例2**: xformersバージョン不一致
```
ERROR: xformers 0.0.33.post2 requires torch==2.9.1, but you have torch 2.9.0+cu126
```

**解決方法**: セル5が自動的に修正します
- 🔍 CUDAバージョンを自動検出（12.6 or 12.8）
- 📦 xformers 0.0.33.post2に合わせてPyTorch 2.9.1に統一
- ✅ torch, torchvision, torchaudio, xformersをすべて互換バージョンで統一
- 🔄 環境チェック機能付き（2回目以降も自動修正）

**推奨構成**:
- CUDA 12.8: `torch==2.9.1`, `torchvision==0.24.1`, `torchaudio==2.9.1`, `xformers==0.0.33.post2`
- CUDA 12.6: `torch==2.9.1`, `torchvision==0.24.1`, `torchaudio==2.9.1`, `xformers==0.0.33.post2`

### ❌ xformersインストールエラー
```
ERROR: Could not find a version that satisfies the requirement xformers
```

**解決方法**: セル5で自動的に対応
- xformers==0.0.33.post2を明示的にインストール
- PyTorch 2.9.0と互換性のあるバージョン

### ⚠️ GPU メモリ不足
```
CUDA out of memory
```

**解決方法**:
1. セル11のワークフロー設定を変更:
   ```python
   "length": 33  # → 25 (フレーム数を減らす)
   "steps": 30   # → 20 (サンプリングステップを減らす)
   ```
2. Google Colabランタイムを再起動
3. 解像度を下げる: `832x480` → `640x360`

### ⚠️ サーバー起動失敗
```
Cloudflare URLの自動取得に失敗しました
```

**解決方法**:
1. セル8を再実行
2. 手動でトンネルを作成:
   ```python
   !./cloudflared-linux-amd64 tunnel --url http://localhost:8188
   ```
3. ポート8188が使用中か確認:
   ```python
   !netstat -tuln | grep 8188
   ```

### ⚠️ AnimateDiffモデル不足エラー
```
[AnimateDiffEvo] - ERROR - No motion models found. Please download one and place in: 
['/content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models', 
 '/content/ComfyUI/models/animatediff_models']
```

**原因**: AnimateDiffのモーションモデルがダウンロードされていない

**解決方法**:
1. セル7を再実行（自動でダウンロードされます）
2. 手動でダウンロード:
   ```python
   !mkdir -p /content/ComfyUI/models/animatediff_models
   !wget -q --show-progress -O /content/ComfyUI/models/animatediff_models/mm_sd15_v3.safetensors \
     https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd15_v3.safetensors
   ```
3. Google Drive使用時は自動的にリンクされます

### ⚠️ モデルダウンロード失敗
```
wget: unable to resolve host address
```

**解決方法**:
1. ネット接続を確認
2. セル7を再実行
3. 手動でダウンロード:
   ```python
   !wget -O /content/drive/MyDrive/ComfyUI_Models/diffusion_models/wan2.1_t2v_1.3B_fp16.safetensors \
     https://huggingface.co/wanxiang/wan2.1_t2v_1.3B/resolve/main/wan2.1_t2v_1.3B_fp16.safetensors
   ```

### ⚠️ JSONアップロード失敗
```
ファイルがアップロードされませんでした
```

**解決方法**:
1. セル10.5を再実行
2. ファイル選択ダイアログが表示されるまで待つ
3. ファイルサイズを確認（10MB以下推奨）
4. JSON形式が正しいか確認:
   ```json
   [
     {
       "scene": 1,
       "timestamp": "0:00-0:05",
       "prompt": "...",
       "negative_prompt": "..."
     }
   ]
   ```

---

## 💡 モデル管理のヒント

### Google Driveを使う場合（推奨）
✅ **メリット**:
- モデルが永続化（セッション終了後も保存）
- 2回目以降は数秒でセットアップ完了
- 複数ノートブックで同じモデルを共有可能

📁 **保存場所**: `/content/drive/MyDrive/ComfyUI_Models/`
```
ComfyUI_Models/
├── diffusion_models/
│   └── wan2.1_t2v_1.3B_fp16.safetensors (3GB)
├── text_encoders/
│   └── umt5_xxl_fp8_e4m3fn_scaled.safetensors (2GB)
├── vae/
│   └── wan_2.1_vae.safetensors (1GB)
└── animatediff_models/
    └── mm_sd15_v3.safetensors (1.7GB)
```

### Google Driveを使わない場合
- モデルはローカルにダウンロード
- セッション終了後は削除される
- 次回また再ダウンロードが必要（5-10分）

---

## ⚙️ カスタマイズオプション

### 動画設定（セル11の`workflow`内）
- **解像度**: `width`/`height` を変更
  - 例: 832x480 → 1280x720 (HD)
  - 例: 832x480 → 640x360 (GPU節約)
  
- **フレーム数**: `length` を変更
  - 例: 33 → 25 (約1.5秒@16fps、GPU節約)
  - 例: 33 → 49 (約3秒@16fps)
  
- **FPS**: `fps` を変更
  - 例: 16 → 24 (より滑らか)
  - 例: 16 → 12 (容量節約)

### 品質設定（セル11の`KSampler`内）
- **ステップ数**: `steps` を増やす/減らす
  - 例: 30 → 20 (高速化、GPU節約)
  - 例: 30 → 40 (高品質、時間増加)
  
- **CFG**: `cfg` を調整
  - 例: 6.0 → 7.5 (プロンプト忠実度↑)
  - 例: 6.0 → 5.0 (より自然な動き)

---

## 🔗 参考リンク
- [ComfyUI GitHub](https://github.com/comfyanonymous/ComfyUI)
- [WAN (Wanxiang) Models](https://huggingface.co/wanxiang)
- [Google Colab Documentation](https://colab.research.google.com/)

---

## 📊 リソース使用量の目安

| 項目 | 必要量 | 備考 |
|------|--------|------|
| **GPU VRAM** | 10-12GB | Tesla T4 (15GB) で動作可能 |
| **Google Drive** | 約6GB | モデル保存用（初回のみ） |
| **Colabディスク** | 約2GB | ComfyUI本体＋カスタムノード |
| **RAM** | 約8GB | 動画生成時 |

---

## ⏱️ 処理時間の目安

| 処理 | 初回 | 2回目以降 (Google Drive) |
|------|------|--------------------------|
| **モデルDL** | 5-10分 | **数秒（DL不要）** |
| **セットアップ** | 15分 | **5分** |
| **動画生成** | 2-3分/シーン | 2-3分/シーン |

---

## ✅ 依存関係のバージョン（推奨・自動構成）

**セル5が自動的に以下のバージョンに統一します**:

### CUDA 12.8環境（Google Colab標準）
- Python: 3.10+
- PyTorch: **2.9.1** (CUDA 12.8対応)
- torchvision: **0.24.1**
- torchaudio: **2.9.1**
- CUDA: **12.8**
- xformers: **0.0.33.post2**

### CUDA 12.6環境
- Python: 3.10+
- PyTorch: **2.9.1** (xformers互換)
- torchvision: **0.24.1**
- torchaudio: **2.9.1**
- CUDA: **12.6**
- xformers: **0.0.33.post2**

**重要**: xformers 0.0.33.post2はPyTorch 2.9.1を要求するため、すべての環境でPyTorch 2.9.1に統一されます。

**注**: セル5が環境を検出して自動的に最適なバージョンをインストールします